In [5]:
import pandas as pd
import os
import graphein
import pickle
path = '../datasets/ALPHAFOLD PDBs/'


## create a df with mutation name and path to che .pdb file

In [6]:

pdb_files = []
for folder in os.listdir(path):
    # if the folder is a directory
    if os.path.isdir(path + folder):
        # for every file in the folder
        for file in os.listdir(path + folder):
            if 'rank_001' in file and file.endswith('.pdb'):
                pdb_files.append(path + folder + '/' + file)



In [7]:
pdb_files

['../datasets/ALPHAFOLD PDBs/G185R_e6971/G185R_e6971_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/E168L_0f4af/E168L_0f4af_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/R53Q_8ee59/R53Q_8ee59_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/W97C_b9ccb/W97C_b9ccb_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/K353Q_ed31e/K353Q_ed31e_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/G360A_4d8ad/G360A_4d8ad_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/G360R_057f7/G360R_057f7_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/A267V_1c2c3/A267V_1c2c3_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/G251D_5ea06/G251D_5ea06_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/

In [8]:
# for each pdb file in pdb_files, get name of the mutation which is the name of the folder before '_'
mutations = []
for pdb in pdb_files:
    mutations.append(pdb.split('/')[-2].split('_')[0])

    

In [9]:
mutations

['G185R',
 'E168L',
 'R53Q',
 'W97C',
 'K353Q',
 'G360A',
 'G360R',
 'A267V',
 'G251D',
 'L116P',
 'T167I',
 'F227S',
 'A122V',
 'Y40S',
 'G362E',
 'E168K',
 'S150L',
 'G361R',
 'G152R',
 'S47L',
 'L44F',
 'I216T',
 'E401Q',
 'E178G',
 'Y272C',
 'P332R',
 'G198D',
 'K248E',
 'R330S',
 'G161R',
 'G170S',
 'M368V',
 'P230S',
 'C120F',
 'I346T',
 'W97G',
 'D18Y',
 'V300G',
 'F73L',
 'G205D',
 'G170A',
 'R225P',
 'G270R',
 'G123A']

In [10]:
# create a dataframe with the mutation and the pdb file

df = pd.DataFrame({'mutation': mutations, 'pdb_file': pdb_files})

out_path = "../datasets/pdb_files.csv"
if not os.path.isfile(out_path):
    df.to_csv(out_path, index=False)


## Parse the .pdb files

In [11]:
from Bio.PDB import PDBParser   
structures = {}
parser = PDBParser()
for i, row in df.iterrows():
    structures[row['mutation']] = parser.get_structure(row['mutation'], row['pdb_file'])
 

In [12]:
structures

{'G185R': <Structure id=G185R>,
 'E168L': <Structure id=E168L>,
 'R53Q': <Structure id=R53Q>,
 'W97C': <Structure id=W97C>,
 'K353Q': <Structure id=K353Q>,
 'G360A': <Structure id=G360A>,
 'G360R': <Structure id=G360R>,
 'A267V': <Structure id=A267V>,
 'G251D': <Structure id=G251D>,
 'L116P': <Structure id=L116P>,
 'T167I': <Structure id=T167I>,
 'F227S': <Structure id=F227S>,
 'A122V': <Structure id=A122V>,
 'Y40S': <Structure id=Y40S>,
 'G362E': <Structure id=G362E>,
 'E168K': <Structure id=E168K>,
 'S150L': <Structure id=S150L>,
 'G361R': <Structure id=G361R>,
 'G152R': <Structure id=G152R>,
 'S47L': <Structure id=S47L>,
 'L44F': <Structure id=L44F>,
 'I216T': <Structure id=I216T>,
 'E401Q': <Structure id=E401Q>,
 'E178G': <Structure id=E178G>,
 'Y272C': <Structure id=Y272C>,
 'P332R': <Structure id=P332R>,
 'G198D': <Structure id=G198D>,
 'K248E': <Structure id=K248E>,
 'R330S': <Structure id=R330S>,
 'G161R': <Structure id=G161R>,
 'G170S': <Structure id=G170S>,
 'M368V': <Structu

## Create graphs from the .pdb files

In [13]:
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.graphs import construct_graph

config = ProteinGraphConfig()
(config.dict())

{'granularity': 'CA',
 'keep_hets': [],
 'insertions': True,
 'alt_locs': 'max_occupancy',
 'pdb_dir': None,
 'verbose': False,
 'exclude_waters': True,
 'deprotonate': False,
 'protein_df_processing_functions': None,
 'edge_construction_functions': [<function graphein.protein.edges.distance.add_peptide_bonds(G: 'nx.Graph') -> 'nx.Graph'>],
 'node_metadata_functions': [<function graphein.protein.features.nodes.amino_acid.meiler_embedding(n: str, d: Dict[str, Any], return_array: bool = False) -> Union[pandas.core.series.Series, numpy.ndarray]>],
 'edge_metadata_functions': None,
 'graph_metadata_functions': None,
 'get_contacts_config': None,
 'dssp_config': None}

In [14]:
graphs = {}
for i, row in df.iterrows():
    print(row['mutation'])
    graphs[row['mutation'] ] = construct_graph(path = row['pdb_file'], config= config, )

Output()

[04/29/24 09:45:17] DEBUG    Deprotonating protein. This removes H atoms from the pdb_df dataframe    ]8;id=944672;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=365391;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#188\188]8;;\

                    DEBUG    Detected 445 total nodes                                                 ]8;id=129847;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=803312;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#435\435]8;;\

G185R


Output()

...

                    DEBUG    Detected 445 total nodes                                                 ]8;id=653986;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=434577;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#435\435]8;;\

Output()

G123A


                    DEBUG    Deprotonating protein. This removes H atoms from the pdb_df dataframe    ]8;id=185466;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=38469;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#188\188]8;;\

                    DEBUG    Detected 445 total nodes                                                 ]8;id=290624;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=413581;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#435\435]8;;\

In [15]:
graphs

{'G185R': <networkx.classes.graph.Graph at 0x105414850>,
 'E168L': <networkx.classes.graph.Graph at 0x331d8a8f0>,
 'R53Q': <networkx.classes.graph.Graph at 0x331d8b2b0>,
 'W97C': <networkx.classes.graph.Graph at 0x331df7010>,
 'K353Q': <networkx.classes.graph.Graph at 0x331d8bcd0>,
 'G360A': <networkx.classes.graph.Graph at 0x331df79a0>,
 'G360R': <networkx.classes.graph.Graph at 0x3332f73a0>,
 'A267V': <networkx.classes.graph.Graph at 0x3332f4970>,
 'G251D': <networkx.classes.graph.Graph at 0x3341bed10>,
 'L116P': <networkx.classes.graph.Graph at 0x3341be4a0>,
 'T167I': <networkx.classes.graph.Graph at 0x3341bfa60>,
 'F227S': <networkx.classes.graph.Graph at 0x3341bf580>,
 'A122V': <networkx.classes.graph.Graph at 0x3345b7400>,
 'Y40S': <networkx.classes.graph.Graph at 0x334abd7e0>,
 'G362E': <networkx.classes.graph.Graph at 0x3345b6950>,
 'E168K': <networkx.classes.graph.Graph at 0x334abe9b0>,
 'S150L': <networkx.classes.graph.Graph at 0x334abf940>,
 'G361R': <networkx.classes.graph.

In [16]:
from graphein.protein.visualisation import plotly_protein_structure_graph

p = plotly_protein_structure_graph(
    graphs['G161R'],
    colour_edges_by="kind",
    colour_nodes_by="degree",
    label_node_ids=False,
    plot_title="Peptide backbone graph. Nodes coloured by degree.",
    node_size_multiplier=1
    )
p.show()

In [17]:
# plot structures['G161R'] 

import nglview as nv
view = nv.show_biopython(structures['G161R'])
view


NGLWidget()

In [18]:
# save the graphs
os.makedirs('../datasets/graphs/', exist_ok=True)
for i, graph in enumerate(graphs):
    mut = df.iloc[i]['mutation']
    with open(f'../datasets/graphs/graph_{mut}.gpickle', 'wb') as f:
        pickle.dump(graph, f)

## Associate to each patient its mutations

In [19]:
import pandas as pd
df_patients =pd.read_excel('../datasets/aku_prin_v2.0.xlsx')

In [20]:
df_patients = df_patients[['Protein change allele 1 ', 'Protein change allele 2']]

In [21]:
df_patients['graph_allele1'] = [graphs[mut] if mut in graphs else None for mut in df_patients['Protein change allele 1 '] ]
df_patients['graph_allele2'] = [graphs[mut] if mut in graphs else None for mut in df_patients['Protein change allele 2'] ]
df_patients['structure_allele1'] = [structures[mut] if mut in structures else None for mut in df_patients['Protein change allele 1 '] ] 
df_patients['structure_allele2'] = [structures[mut] if mut in structures else None for mut in df_patients['Protein change allele 2'] ]

In [22]:
#show all rows
pd.set_option('display.max_columns', 500)
df_patients

,Protein change allele 1,Protein change allele 2,graph_allele1,graph_allele2,structure_allele1,structure_allele2
0,G161R,His371Profs,"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...",None,"(((<Residue MET het= resseq=1 icode= >, <Resi...",None
1,G161R,G161R,"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(((<Residue MET het= resseq=1 icode= >, <Resi...","(((<Residue MET het= resseq=1 icode= >, <Resi..."
2,G161R,M368V,"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(((<Residue MET het= resseq=1 icode= >, <Resi...","(((<Residue MET het= resseq=1 icode= >, <Resi..."
3,Y6_G29del ivs1-1G>A,Y6_G29del ivs1-1G>A,None,None,None,None
4,G161R,G161R,"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(((<Residue MET het= resseq=1 icode= >, <Resi...","(((<Residue MET het= resseq=1 icode= >, <Resi..."
...,...,...,...,...,...,...
214,G251D,del ex13 (MLPA),"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...",None,"(((<Residue MET het= resseq=1 icode= >, <Resi...",None
215,E401Q,(p.(Ala218_Asn219insLysIle)),"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...",None,"(((<Residue MET het= resseq=1 icode= >, <Resi...",None
216,R53Q,R53Q,"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(((<Residue MET het= resseq=1 icode= >, <Resi...","(((<Residue MET het= resseq=1 icode= >, <Resi..."
217,ivs8-2A>G (R184fs),n.i. (MLPAneg),None,None,None,None


In [23]:
df_patients.to_csv('../datasets/aku_prin_v2.0_with_graphs.csv', index=False)



In [24]:
#read the csv file
df_patients_2 = pd.read_csv('../datasets/aku_prin_v2.0_with_graphs.csv')


In [25]:
graph_test = df_patients_2['graph_allele1'][0]
graph_test_og = df_patients['graph_allele1'][0]

In [28]:
type(graph_test_og)

networkx.classes.graph.Graph

In [84]:
import networkx as nx

def string_to_graph(input_string):
    # Initialize a NetworkX graph
    G = nx.Graph()

    # Split the input string into lines
    lines = input_string.strip().split('\n')

    # Iterate through each line
    for line in lines:
        # Split the line into nodes and edges
        nodes = line.strip().split()
        # Assuming the first element is a node and the rest are its neighbors
        node = nodes[0]
        neighbors = nodes[1:]

        # Add the node to the graph
        G.add_node(node)

        # Add edges between the node and its neighbors
        for neighbor in neighbors:
            G.add_edge(node, neighbor)

    return G

In [89]:
type(graph_test)
# convert to a scipy sparse matrix
graph = string_to_graph(graph_test)
type(graph)

networkx.classes.graph.Graph

In [93]:
graph.edges

EdgeView([('Graph', 'named'), ('Graph', "'G161R_c8ba5_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000'"), ('Graph', 'with'), ('Graph', '445'), ('Graph', 'nodes'), ('Graph', 'and'), ('Graph', '444'), ('Graph', 'edges')])

In [ ]:
from torch_geometric.data import Data


In [4]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [7]:
import logging, sys
logging.disable(sys.maxsize)

from utils import create_graph_df
df_p = create_graph_df()
df_p

[04/29/24 11:01:57] DEBUG    Deprotonating protein. This removes H atoms from the pdb_df dataframe    ]8;id=986137;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=712395;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#188\188]8;;\

                    DEBUG    Detected 445 total nodes                                                 ]8;id=555293;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=834650;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#435\435]8;;\

                    DEBUG    Deprotonating protein. This removes H atoms from the pdb_df dataframe    ]8;id=410997;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=663037;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#188\188]8;;\

                    DEBUG    Detected 445 total nodes                                                 ]8;id=263213;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=691651;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#435\435]8;;\

                    DEBUG    Deprotonating protein. This removes H atoms from the pdb_df dataframe    ]8;id=810575;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=807749;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#188\188]8;;\

...

                    DEBUG    Deprotonating protein. This removes H atoms from the pdb_df dataframe    ]8;id=501819;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=446960;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#188\188]8;;\

                    DEBUG    Detected 445 total nodes                                                 ]8;id=905237;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=286985;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#435\435]8;;\

                    DEBUG    Deprotonating protein. This removes H atoms from the pdb_df dataframe    ]8;id=24761;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=707491;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#188\188]8;;\

                    DEBUG    Detected 445 total nodes                                                 ]8;id=291607;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=258807;file:///Users/lavo/Desktop/Projects/CHL-Project/.venv/lib/python3.10/site-packages/graphein/protein/graphs.py#435\435]8;;\

,Protein change allele 1,Protein change allele 2,graph_allele1,graph_allele2,structure_allele1,structure_allele2
0,G161R,His371Profs,"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...",None,"(((<Residue MET het= resseq=1 icode= >, <Resi...",None
1,G161R,G161R,"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(((<Residue MET het= resseq=1 icode= >, <Resi...","(((<Residue MET het= resseq=1 icode= >, <Resi..."
2,G161R,M368V,"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(((<Residue MET het= resseq=1 icode= >, <Resi...","(((<Residue MET het= resseq=1 icode= >, <Resi..."
3,Y6_G29del ivs1-1G>A,Y6_G29del ivs1-1G>A,None,None,None,None
4,G161R,G161R,"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(((<Residue MET het= resseq=1 icode= >, <Resi...","(((<Residue MET het= resseq=1 icode= >, <Resi..."
...,...,...,...,...,...,...
214,G251D,del ex13 (MLPA),"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...",None,"(((<Residue MET het= resseq=1 icode= >, <Resi...",None
215,E401Q,(p.(Ala218_Asn219insLysIle)),"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...",None,"(((<Residue MET het= resseq=1 icode= >, <Resi...",None
216,R53Q,R53Q,"(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(A:MET:1, A:ALA:2, A:GLU:3, A:LEU:4, A:LYS:5, ...","(((<Residue MET het= resseq=1 icode= >, <Resi...","(((<Residue MET het= resseq=1 icode= >, <Resi..."
217,ivs8-2A>G (R184fs),n.i. (MLPAneg),None,None,None,None


In [2]:
type(df_p['graph_allele1'][0])

networkx.classes.graph.Graph